In [1]:
print("hello")

hello


In [2]:
from mip import *

MODULE 1

In [ ]:
def module1(GPh,disutility,t,E,SD,IE,P1,MinC,MaxC,SP,RQ,beta,Lmax):
    """
    GPh = [0.18 for _ in range(24)]
    GPh[10] = 0
    disutility = 0.1
    t = 3  # appliance.t
    E = 0.8  # storage.E
    IE = 2.6 # storage.IE
    SD = 1 # storage.sd
    P1 = 1800  # household.p
    MinC = 2.6 # min storage
    MaxC = 6.4 # max storage
    SP = 1
    RQ = [1.3 for _ in range(24)]  # stockage.RQ
    beta = 12 # appliance.beta
    Lmax = 6000
    """
    m = Model(sense=MINIMIZE,solver_name=CBC)
    
    GE = [ m.add_var(name='GE'+str(i),ub=Lmax,lb=0) for i in range(1,25)]
    S = [ m.add_var(name='S'+str(i),var_type=BINARY) for i in range(1,25)]
    IC = [ m.add_var(name='IC'+str(i),var_type=BINARY) for i in range(1,25)]
    BE = [ m.add_var(name='BE'+str(i)) for i in range(1,25)]
    RE = [ m.add_var(name='RE'+str(i),ub=RQ[i-1]) for i in range(1,25) ]
    SE = [ m.add_var(name='SE'+str(i),ub=MaxC,lb=MinC) for i in range(1,25)]
    tau = m.add_var(name='Tau',ub=beta)
    for h in range(0,24):
        m+= (S[h]*P1 + IC[h]*SP == GE[h] + BE[h] + RE[h]) 
    
    #Contrainte 4
    m+= SE[0] == IE*SD + IC[0]*SP*E-BE[0]
    
    for h in range(1,24):
        m+= SE[h] == SE[h-1]*SD  + IC[h]*SP*E-BE[h]
    
    #Contrainte 8
    m+= xsum(S[h] for h in range(24)) == t
    #Contrainte 11
    for h in range(24):
        m+= (h+1)*S[h] <= tau
    print("End constraint")
    
    m.objective = xsum(GPh[h] * GE[h] for h in range(24)) + disutility*tau

    m.max_gap = 0.01
    status = m.optimize(max_seconds=30)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(m.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        print('solution:')
    if status == OptimizationStatus.INFEASIBLE:
        print("Infeasible")
    dsk = []
    
    for i in range(24):
        dsk.append(m.vars['S'+str(i+1)].x*P1 + m.vars['IC'+str(i+1)].x*SP - RQ[i]-
                   m.vars['SE'+str(i+1)].x-m.vars['BE'+str(i+1)].x+MinC)
    return m.objective_bound,dsk


MODULE 2

In [39]:

def module2(GPh,disutility,t,E,SD,IE,P1,MinC,MaxC,SP,RQ,beta,Lmax,CK_noTrade_1,ME_1
            ,disutility2,t2,E2,SD2,IE2,P2,MinC2,MaxC2,SP2,RQ2,beta2,CK_noTrade_2,ME_2):
    """
    GPh = [0.18 for _ in range(24)]
    GPh[10] = 0
    disutility = 0.1
    t = 3  # appliance.t
    E = 0.8  # storage.E
    IE = 2.6 # storage.IE
    SD = 1 # storage.sd
    P1 = 1800  # household.p
    P2 = 1800
    MinC = 2.6 # min storage
    MaxC = 6.4 # max storage
    SP = 1
    RQ = [1.3 for _ in range(24)]  # stockage.RQ
    beta = 12 # appliance.beta
    Lmax = 6000
    CK_noTrade_1 = 900
    CK_noTrade_2 = 900
    ME_1 = [w for w in range(24)]#DSK_1
    ME_2 = [w for w in range(24)]#DSK_2
    """
    
    m = Model(sense=MINIMIZE,solver_name=CBC)
    
    MP = [ m.add_var(name='MP'+str(i),ub=GPh[i-1],lb=0) for i in range(1,25)]
    
    GE_1 = [ m.add_var(name='GE_1'+str(i),ub=Lmax,lb=0) for i in range(1,25)]
    S_1 = [ m.add_var(name='S_1'+str(i),var_type=BINARY) for i in range(1,25)]
    IC_1 = [ m.add_var(name='IC_1'+str(i),var_type=BINARY) for i in range(1,25)]
    BE_1 = [ m.add_var(name='BE_1'+str(i)) for i in range(1,25)]
    RE_1 = [ m.add_var(name='RE_1'+str(i),ub=RQ[i-1]) for i in range(1,25) ]
    SE_1 = [ m.add_var(name='SE_1'+str(i),ub=MaxC,lb=MinC) for i in range(1,25)]
    
    tau_1 = m.add_var(name='Tau_1',ub=beta)
    
    GE_2 = [ m.add_var(name='GE_2'+str(i),ub=Lmax,lb=0) for i in range(1,25)]
    S_2 = [ m.add_var(name='S_2'+str(i),var_type=BINARY) for i in range(1,25)]
    IC_2 = [ m.add_var(name='IC_2'+str(i),var_type=BINARY) for i in range(1,25)]
    BE_2 = [ m.add_var(name='BE_2'+str(i)) for i in range(1,25)]
    RE_2 = [ m.add_var(name='RE_2'+str(i),ub=RQ2[i-1]) for i in range(1,25) ]
    SE_2 = [ m.add_var(name='SE_2'+str(i),ub=MaxC2,lb=MinC2) for i in range(1,25)]
    
    tau_2 = m.add_var(name='Tau_2',ub=beta2)

    print('end')
    #%%
    #Contrainte 4&5
    m+= SE_1[0] == IE*SD + IC_1[0]*SP*E-BE_1[0]
    m+= SE_2[0] == IE2*SD2 + IC_2[0]*SP2*E2-BE_1[0]
    
    for h in range(1,24):
        m+= SE_1[h] == SE_1[h-1]*SD  + IC_1[h]*SP*E-BE_1[h]
        m+= SE_2[h] == SE_2[h-1]*SD2  + IC_2[h]*SP2*E2-BE_2[h]
    #Contrainte 8
    m+= xsum(S_1[h] for h in range(24)) == t
    m+= xsum(S_2[h] for h in range(24)) == t2
    #Contrainte 11
    for h in range(24):
        m+= (h+1)*S_1[h] <= tau_1
        m+= (h+1)*S_2[h] <= tau_2
        
    #Constrain 21
    for h in range(0,24):
            m+= (S_1[h]*P1 + IC_1[h]*SP <= GE_1[h] + BE_1[h] + RE_1[h] + ME_1[h]) 
            m+= (S_2[h]*P2 + IC_2[h]*SP2 <= GE_2[h] + BE_2[h] + RE_2[h] + ME_2[h]) 
    #Constrain 24
    m+= xsum( GPh[h] * GE_1[h] + MP[h] * ME_1[h]  for h in range(24)) + disutility*tau_1 <= CK_noTrade_1
    m+= xsum(GPh[h] * GE_2[h]+ MP[h]*ME_2[h] for h in range(24)) + disutility2*tau_2 <= CK_noTrade_2
    
    
    print("End constraint")
    #%%
    m.objective = xsum(GPh[h] * GE_1[h] + GPh[h] * GE_2[h] for h in range(24)) + disutility*tau_1 + disutility*tau_2
    m.max_gap = 0.01
    status = m.optimize(max_seconds=30)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(m.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        print('solution:')
    if status == OptimizationStatus.INFEASIBLE:
                   print("Infeasible")
    MP_out = []
    for i in range(24):
        MP_out.append(m.vars['MP'+str(i+1)].x)
    return MP_out

MODULE 3

In [67]:
def module3(GPh,disutility1,t1,E1,SD1,IE1,P1,MinC1,MaxC1,SP1,RQ1,beta1,Lmax,CK_noTrade_1,MP
            ,disutility2,t2,E2,SD2,IE2,P2,MinC2,MaxC2,SP2,RQ2,beta2,CK_noTrade_2):
    m = Model(sense=MINIMIZE,solver_name=CBC)
    
    
    
    GE_1 = [ m.add_var(name='GE_1'+str(i),ub=Lmax,lb=0) for i in range(1,25)]
    S_1 = [ m.add_var(name='S_1'+str(i),var_type=BINARY) for i in range(1,25)]
    IC_1 = [ m.add_var(name='IC_1'+str(i),var_type=BINARY) for i in range(1,25)]
    BE_1 = [ m.add_var(name='BE_1'+str(i)) for i in range(1,25)]
    RE_1 = [ m.add_var(name='RE_1'+str(i),ub=RQ1[i-1]) for i in range(1,25) ]
    SE_1 = [ m.add_var(name='SE_1'+str(i),ub=MaxC1,lb=MinC1) for i in range(1,25)]
    ME_1 = [ m.add_var(name='ME1'+str(i)) for i in range(1,25)]
    tau_1 = m.add_var(name='Tau_1',ub=beta1)
    MQ_1 = [ m.add_var(name='MQ1'+str(i)) for i in range(1,25)]
    
    GE_2 = [ m.add_var(name='GE_2'+str(i),ub=Lmax,lb=0) for i in range(1,25)]
    S_2 = [ m.add_var(name='S_2'+str(i),var_type=BINARY) for i in range(1,25)]
    IC_2 = [ m.add_var(name='IC_2'+str(i),var_type=BINARY) for i in range(1,25)]
    BE_2 = [ m.add_var(name='BE_2'+str(i)) for i in range(1,25)]
    RE_2 = [ m.add_var(name='RE_2'+str(i),ub=RQ2[i-1]) for i in range(1,25) ]
    SE_2 = [ m.add_var(name='SE_2'+str(i),ub=MaxC2,lb=MinC2) for i in range(1,25)]
    ME_2 = [ m.add_var(name='ME2'+str(i)) for i in range(1,25)]
    tau_2 = m.add_var(name='Tau_2',ub=beta2)
    MQ_2 = [ m.add_var(name='MQ2'+str(i)) for i in range(1,25)]
    
    print('end')
    #%%
    #Contrainte 4&5
    m+= SE_1[0] == IE1*SD1 + IC_1[0]*SP1*E1-BE_1[0]
    m+= SE_2[0] == IE2*SD2 + IC_2[0]*SP2*E2-BE_1[0]
    
    for h in range(1,24):
        m+= SE_1[h] == SE_1[h-1]*SD1 + IC_1[h]*SP1*E1-BE_1[h]
        m+= SE_2[h] == SE_2[h-1]*SD2  + IC_2[h]*SP2*E2-BE_2[h]
    #Contrainte 8
    m+= xsum(S_1[h] for h in range(24)) == t1
    m+= xsum(S_2[h] for h in range(24)) == t2
    #Contrainte 11
    for h in range(24):
        m+= (h+1)*S_1[h] <= tau_1
        m+= (h+1)*S_2[h] <= tau_2
        
    #Constrain 25
    for h in range(0,24):
            m+= (S_1[h]*P1 + IC_1[h]*SP1 == GE_1[h] + BE_1[h] + RE_1[h] + ME_1[h]) 
            m+= (S_2[h]*P2 + IC_2[h]*SP2 == GE_2[h] + BE_2[h] + RE_2[h] + ME_2[h]) 
    #Constrain 24
    m+= xsum(GPh[h] * GE_1[h] + MP[h] * ME_1[h]  for h in range(24)) + disutility1*tau_1 <= CK_noTrade_1
    m+= xsum(GPh[h] * GE_2[h]+ MP[h]*ME_2[h] for h in range(24)) + disutility2*tau_2 <= CK_noTrade_2
    
    #Constrain 26
    for h in range(24):
        m+= ME_1[h]  + ME_2[h]  == 0

    #Constrain 27
    for h in range(24):
        m+= MQ_1[h] == (S_1[h]*P1+IC_1[h]*SP1 - GE_1[h] - RQ1[h]-SE_1[h] - BE_1[h]) + MinC1
        m+= MQ_2[h] == (S_2[h]*P2+IC_2[h]*SP2 - GE_2[h] - RQ2[h]-SE_2[h] - BE_2[h]) + MinC2
     

    #Constrain 28
    for h in range(24):
        m+=ME_1[h] >= MQ_1[h]
        m+=ME_2[h] >= MQ_2[h]
   
    m.objective = xsum(GPh[h] * GE_1[h] + GPh[h] * GE_2[h] for h in range(24)) + disutility1*tau_1 + disutility2*tau_2
    
    m.max_gap = 0.01
    status = m.optimize(max_seconds=30)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(m.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        print('solution:')
    if status == OptimizationStatus.INFEASIBLE:
                   print("Infeasible")
    ME_1_out = []
    ME_2_out = []
    for i in range(24):
        ME_1_out.append(m.vars['ME1'+str(i+1)].x)
        ME_2_out.append(m.vars['ME2'+str(i+1)].x)
    return m.objective_bound,[ME_1_out,ME_2_out]
    

In [68]:
# ECO-TRADE Algorithm
GPh = [0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.15,0.15,0.15,0.15,0.15,0.17,0.17,0.17,0.17,0.15,0.15,0.15,0.13,0.13,0.13,0.13]
Lmax = 6000
#initialisation maison1
disutility1 = 0.1
t1 = 3  # appliance.t
E1 = 0.1  # storage.E
IE1 = 2.6 # storage.IE
SD1 = 1 # storage.sd
P1 = 1800  # household.p
MinC1 = 2.6 # min storage
MaxC1 = 6.4 # max storage
SP1 = 1
RQ1 = [1 for _ in range(24)]  # stockage.RQ
beta1 = 12 # appliance.beta
#initialisation maison2
disutility1 = 0.2
t2 = 10  # appliance.t
E2 = 0.1  # storage.E
IE2 = 0 # storage.IE
SD2 = 1 # storage.sd
P2= 800  # household.p
MinC2 = 0 # min storage
MaxC2 = 10 # max storage
SP2 = 0.1
RQ2 = [0.1 for _ in range(24)]  # stockage.RQ
beta2 = 23 # appliance.beta



epsilon_count, epsilon_max, Cpre = 0, 0, 0
Ccur, epsilon_cur, epsilon = 1, 0, 0.7
CK_NoTrade_1,DSK1 = module1(GPh,disutility1,t1,E1,SD1,IE1,P1,MinC1,MaxC1,SP1,RQ1,beta1,Lmax)
CK_NoTrade_2,DSK2 = module1(GPh,disutility1,t2,E2,SD2,IE2,P2,MinC2,MaxC2,SP2,RQ2,beta2,Lmax)
Cpre = CK_NoTrade_1 + CK_NoTrade_2

print("Cost at initialization :", Cpre)
i = 1
while epsilon_count <= epsilon_max:
    print("*****************************"+str(i)+"**************")
    i+=1
    
    prix = module2(GPh,disutility1,t1,E1,SD1,IE1,P1,MinC1,MaxC1,SP1,RQ1,beta1,Lmax,CK_NoTrade_1,DSK1
            ,disutility1,t2,E2,SD2,IE2,P2,MinC2,MaxC2,SP2,RQ2,beta2,CK_NoTrade_2,DSK2)
    Ccur, fin = module3(GPh,disutility1,t1,E1,SD1,IE1,P1,MinC1,MaxC1,SP1,RQ1,beta1,Lmax,CK_NoTrade_1,prix
            ,disutility1,t2,E2,SD2,IE2,P2,MinC2,MaxC2,SP2,RQ2,beta2,CK_NoTrade_2)
    epsilon_cur = abs((Cpre - Ccur)/Cpre)
    if epsilon_cur <= epsilon:
        epsilon_count += 1
    else:
        epsilon_count = 0
    Cpre = Ccur
# final solution :
print("Solution :", fin)


"""
print("Cost at initialization :", Cpre)
while epsilon_count <= epsilon_max:
    module2()
    Ccur, x = module3()

    epsilon_cur = abs((Cpre - Ccur)/Cpre)
    if epsilon_cur <= epsilon:
        epsilon_count += 1
    else:
        epsilon_count = 0
    Cpre = Ccur

# final solution :
print("Solution :", x)
"""

End constraint
optimal solution cost 702.21 found
solution:
End constraint
optimal solution cost 1044.2544 found
solution:
Cost at initialization : 1746.4124022410156
*****************************1**************
end
End constraint
optimal solution cost 5.0186 found
solution:
end
Infeasible


TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'